# Import important libraries

In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image
import cv2
import matplotlib.pyplot as plt
import re
import numpy as np
from PIL import Image as PILImage
import easyocr


In [ ]:
# Initialize EasyOCR reader (this should be done once for efficiency)
reader = easyocr.Reader(['ar'], gpu=False)

# Face of id card without need to rotate .


In [24]:
# Function to preprocess the cropped image
def preprocess_image(cropped_image):
    gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    threshold_image = cv2.adaptiveThreshold(blurred_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 17, 2)
    return threshold_image

# Functions for specific fields with custom OCR configurations
def extract_text(image, bbox, lang='ara'):
    x1, y1, x2, y2 = bbox
    cropped_image = image[y1:y2, x1:x2]
    preprocessed_image = preprocess_image(cropped_image)
    results = reader.readtext(preprocessed_image, detail=0, paragraph=True)
    text = ' '.join(results)
    return text.strip()

# Function to detect national ID numbers in a cropped image
def detect_national_id(cropped_image):
    model = YOLO('detect_id.pt')  # Load the model directly in the function
    results = model(cropped_image)
    detected_info = []

    for result in results:
        for box in result.boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            detected_info.append((cls, x1))
            cv2.rectangle(cropped_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(cropped_image, str(cls), (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    detected_info.sort(key=lambda x: x[1])
    id_number = ''.join([str(cls) for cls, _ in detected_info])
    
    return id_number

# Function to remove numbers from a string
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

# Function to plot image with bounding boxes
def plot_image_with_boxes(image, boxes):
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    ax = plt.gca()
    for box in boxes:
        x1, y1, x2, y2 = [int(coord) for coord in box]
        rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, color='red', linewidth=2)
        ax.add_patch(rect)
    plt.show()

# Function to expand bounding box height only
def expand_bbox_height(bbox, scale=1.2, image_shape=None):
    x1, y1, x2, y2 = bbox
    width = x2 - x1
    height = y2 - y1
    center_x = x1 + width // 2
    center_y = y1 + height // 2
    new_height = int(height * scale)
    new_y1 = max(center_y - new_height // 2, 0)
    new_y2 = min(center_y + new_height // 2, image_shape[0])
    return [x1, new_y1, x2, new_y2]

# Function to process the cropped image
def process_image(cropped_image):
    # Load the trained YOLO model for objects (fields) detection
    model = YOLO('detect_odjects.pt')
    results = model(cropped_image)

    # Variables to store extracted values
    first_name = ''
    second_name = ''
    merged_name = ''
    nid = ''
    address = ''
    serial = ''

    # Loop through the results
    for result in results:
        result.show()  # Shows the image with bounding boxes
        output_path = 'd2.jpg'
        result.save(output_path)
        display(Image(filename=output_path))

        boxes = [box.xyxy[0].tolist() for box in result.boxes]
        plot_image_with_boxes(cropped_image, boxes)

        for box in result.boxes:
            bbox = box.xyxy[0].tolist()
            class_id = int(box.cls[0].item())
            class_name = result.names[class_id]
            bbox = [int(coord) for coord in bbox]

            if class_name == 'firstName':
                first_name = extract_text(cropped_image, bbox, lang='ara')
            elif class_name == 'lastName':
                second_name = extract_text(cropped_image, bbox, lang='ara')
            elif class_name == 'serial':
                serial = extract_text(cropped_image, bbox, lang='eng')
            elif class_name == 'address':
                address = extract_text(cropped_image, bbox, lang='ara')
                address = remove_numbers(address)
            elif class_name == 'nid':
                expanded_bbox = expand_bbox_height(bbox, scale=1.5, image_shape=cropped_image.shape)
                cropped_nid = cropped_image[expanded_bbox[1]:expanded_bbox[3], expanded_bbox[0]:expanded_bbox[2]]
                nid = detect_national_id(cropped_nid)

    merged_name = f"{first_name} {second_name}"
    print(f"First Name: {first_name}")
    print(f"Second Name: {second_name}")
    print(f"Full Name: {merged_name}")
    print(f"National ID: {nid}")
    print(f"Address: {address}")
    print(f"Serial: {serial}")

    decoded_info = decode_egyptian_id(nid)
    for key, value in decoded_info.items():
        print(f"{key}: {value}")

# Function to decode the Egyptian ID number
def decode_egyptian_id(id_number):
    governorates = {
        '01': 'Cairo',
        '02': 'Alexandria',
        '03': 'Port Said',
        '04': 'Suez',
        '11': 'Damietta',
        '12': 'Dakahlia',
        '13': 'Ash Sharqia',
        '14': 'Kaliobeya',
        '15': 'Kafr El - Sheikh',
        '16': 'Gharbia',
        '17': 'Monoufia',
        '18': 'El Beheira',
        '19': 'Ismailia',
        '21': 'Giza',
        '22': 'Beni Suef',
        '23': 'Fayoum',
        '24': 'El Menia',
        '25': 'Assiut',
        '26': 'Sohag',
        '27': 'Qena',
        '28': 'Aswan',
        '29': 'Luxor',
        '31': 'Red Sea',
        '32': 'New Valley',
        '33': 'Matrouh',
        '34': 'North Sinai',
        '35': 'South Sinai',
        '88': 'Foreign'
    }

    if len(id_number) != 14:
        raise ValueError("ID number must be 14 digits long")

    century_digit = int(id_number[0])
    year = int(id_number[1:3])
    month = int(id_number[3:5])
    day = int(id_number[5:7])
    governorate_code = id_number[7:9]
    gender_code = int(id_number[12:13])

    if century_digit == 2:
        century = "1900-1999"
        full_year = 1900 + year
    elif century_digit == 3:
        century = "2000-2099"
        full_year = 2000 + year
    else:
        raise ValueError("Invalid century digit")

    gender = "Male" if gender_code % 2 != 0 else "Female"
    governorate = governorates.get(governorate_code, "Unknown")
    birth_date = f"{full_year:04d}-{month:02d}-{day:02d}"

    return {
        'Birth Date': birth_date,
        'Governorate': governorate,
        'Gender': gender
    }

# Function to detect the ID card and pass it to the existing code
def detect_and_process_id_card(image_path):
    # Load the ID card detection model
    id_card_model = YOLO('detect_id_card.pt')

    # Perform inference to detect the ID card
    id_card_results = id_card_model(image_path)

    # Load the original image using OpenCV
    image = cv2.imread(image_path)

    # Crop the ID card from the image
    for result in id_card_results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Get bounding box coordinates
            cropped_image = image[y1:y2, x1:x2]

    # Pass the cropped image to the existing processing function
    process_image(cropped_image)

In [ ]:
detect_and_process_id_card('font_ID.jpg')

# Back of id card

In [ ]:
# Load the YOLO models
model_id_card = YOLO('detect_id_card.pt')  # Model for detecting ID cards
model_job = YOLO('detect_odjects.pt')  # Model for extracting objects field

# Function to preprocess the cropped image
def preprocess_image(cropped_image):
    gray_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    threshold_image = cv2.adaptiveThreshold(blurred_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 17, 2)
    return threshold_image

# Functions for specific fields with custom OCR configurations
def extract_text(image, bbox, lang='ara'):
    x1, y1, x2, y2 = bbox
    cropped_image = image[y1:y2, x1:x2]
    preprocessed_image = preprocess_image(cropped_image)
    results = reader.readtext(preprocessed_image, detail=0, paragraph=True)
    text = ' '.join(results)
    return text.strip()

# Function to plot image with bounding boxes
def plot_image_with_boxes(image, boxes):
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    ax = plt.gca()
    for box in boxes:
        x1, y1, x2, y2 = [int(coord) for coord in box]
        rect = plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, color='red', linewidth=2)
        ax.add_patch(rect)
    plt.show()

# Function to detect ID card using the ID card detection model
def detect_id_card(image_path, model):
    results = model(image_path)
    image = cv2.imread(image_path)

    id_card_bboxes = []
    for result in results:
        # Extract bounding boxes
        id_card_bboxes.extend([box.xyxy[0].tolist() for box in result.boxes])

    return image, id_card_bboxes

# Function to process ID card image and extract job field
def process_image(image_path, model_id_card, model_job):
    # Detect ID card
    image, id_card_bboxes = detect_id_card(image_path, model_id_card)

    # Assuming only one ID card detected, process the first one
    if id_card_bboxes:
        bbox = id_card_bboxes[0]  # Get the first detected ID card bbox

        # Crop the ID card region
        x1, y1, x2, y2 = [int(coord) for coord in bbox]
        id_card_image = image[y1:y2, x1:x2]

        # Use the cropped ID card image as input for the job detection model
        results = model_job(id_card_image)  # Pass the cropped image directly

        # Loop through the results
        job = ''
        for result in results:
            # Display the results
            result.show()  # Shows the image with bounding boxes

            # Save the output image
            output_path = 'job_output.jpg'
            result.save(output_path)

            # Visualize the saved output image
            display(Image(filename=output_path))

            # Extract bounding boxes and class names
            boxes = [box.xyxy[0].tolist() for box in result.boxes]
            plot_image_with_boxes(id_card_image, boxes)  # Use id_card_image for plotting

            # Extract text for the "job" field
            for box in result.boxes:
                bbox = box.xyxy[0].tolist()  # Get bounding box coordinates
                class_id = int(box.cls[0].item())  # Get class ID
                class_name = result.names[class_id]  # Get class name

                # Convert bounding box coordinates to integers
                bbox = [int(coord) for coord in bbox]

                if class_name == 'job':
                    job = extract_text(id_card_image, bbox, lang='ara')

        # Print the extracted value
        print(f"Job: {job}")
    else:
        print("No ID card detected")

In [ ]:

process_image('font_ID.jpg', model_id_card, model_job)  # Use the uploaded image